# 📊 版本对比分析报告

> 自动对比多个版本的性能、质量和成本数据

**目的**: 可视化展示优化效果，辅助决策

In [ ]:
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ 模块导入成功")

## 📂 加载基准数据

In [ ]:
def load_all_benchmarks(benchmark_dir: Path) -> pd.DataFrame:
    """
    加载所有基准 JSON 文件，转换为 DataFrame
    """
    benchmark_files = sorted(benchmark_dir.glob("*.json"))
    
    if not benchmark_files:
        print("⚠️  未找到基准数据文件")
        return pd.DataFrame()
    
    records = []
    
    for file in benchmark_files:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 展平嵌套结构
        record = {
            'version': data['version'],
            'timestamp': data['timestamp'],
            'file': file.name
        }
        
        # 性能指标
        for key, value in data.get('performance', {}).items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    record[f'perf_{key}_{sub_key}'] = sub_value
            else:
                record[f'perf_{key}'] = value
        
        # 质量指标
        for key, value in data.get('quality', {}).items():
            record[f'quality_{key}'] = value
        
        # 成本指标
        for key, value in data.get('cost', {}).items():
            record[f'cost_{key}'] = value
        
        records.append(record)
    
    df = pd.DataFrame(records)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    return df

# 加载数据
benchmark_dir = Path.cwd()
df_benchmarks = load_all_benchmarks(benchmark_dir)

if not df_benchmarks.empty:
    print(f"✅ 成功加载 {len(df_benchmarks)} 个版本的基准数据")
    display(df_benchmarks[['version', 'timestamp', 'file']].head())
else:
    print("ℹ️  请先运行 eval_framework.ipynb 生成基准数据")

## ⚡ 性能对比

In [ ]:
if not df_benchmarks.empty:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 端到端响应时间
    if 'perf_end_to_end_avg_time' in df_benchmarks.columns:
        ax = axes[0, 0]
        df_benchmarks.plot(x='version', y='perf_end_to_end_avg_time', 
                          kind='bar', ax=ax, color='skyblue', legend=False)
        ax.set_title('端到端响应时间对比', fontsize=14, fontweight='bold')
        ax.set_ylabel('时间 (秒)', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
    
    # 2. LLM 响应时间
    if 'perf_llm_avg_response_time' in df_benchmarks.columns:
        ax = axes[0, 1]
        df_benchmarks.plot(x='version', y='perf_llm_avg_response_time', 
                          kind='bar', ax=ax, color='lightcoral', legend=False)
        ax.set_title('LLM 响应时间对比', fontsize=14, fontweight='bold')
        ax.set_ylabel('时间 (秒)', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
    
    # 3. 数据获取时间
    if 'perf_data_fetching_avg_time' in df_benchmarks.columns:
        ax = axes[1, 0]
        df_benchmarks.plot(x='version', y='perf_data_fetching_avg_time', 
                          kind='bar', ax=ax, color='lightgreen', legend=False)
        ax.set_title('数据获取时间对比', fontsize=14, fontweight='bold')
        ax.set_ylabel('时间 (秒)', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
    
    # 4. 指标计算时间（总和）
    indicator_cols = [col for col in df_benchmarks.columns if col.startswith('perf_') and 'times_avg' in col]
    if indicator_cols:
        ax = axes[1, 1]
        df_benchmarks['total_indicator_time'] = df_benchmarks[indicator_cols].sum(axis=1)
        df_benchmarks.plot(x='version', y='total_indicator_time', 
                          kind='bar', ax=ax, color='gold', legend=False)
        ax.set_title('指标计算时间对比', fontsize=14, fontweight='bold')
        ax.set_ylabel('时间 (秒)', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('performance_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ 性能对比图表已生成")

## ✅ 质量对比

In [ ]:
if not df_benchmarks.empty:
    quality_cols = [col for col in df_benchmarks.columns if col.startswith('quality_')]
    
    if quality_cols:
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # 创建分组柱状图
        df_quality = df_benchmarks[['version'] + quality_cols].set_index('version')
        df_quality.plot(kind='bar', ax=ax, width=0.8)
        
        ax.set_title('质量指标对比', fontsize=16, fontweight='bold')
        ax.set_ylabel('评分/比率', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.set_ylim(0, 1.1)
        ax.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='及格线 (0.8)')
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('quality_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("✅ 质量对比图表已生成")

## 💰 成本对比

In [ ]:
if not df_benchmarks.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. Token 使用量
    if 'cost_avg_total_tokens' in df_benchmarks.columns:
        ax = axes[0]
        df_benchmarks.plot(x='version', y='cost_avg_total_tokens', 
                          kind='bar', ax=ax, color='mediumpurple', legend=False)
        ax.set_title('平均 Token 使用量', fontsize=14, fontweight='bold')
        ax.set_ylabel('Token 数', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
        
        # 添加数值标签
        for i, v in enumerate(df_benchmarks['cost_avg_total_tokens']):
            ax.text(i, v + 50, f'{int(v)}', ha='center', va='bottom', fontsize=10)
    
    # 2. 单次查询成本
    if 'cost_total_cost_cny' in df_benchmarks.columns:
        ax = axes[1]
        df_benchmarks.plot(x='version', y='cost_total_cost_cny', 
                          kind='bar', ax=ax, color='salmon', legend=False)
        ax.set_title('单次查询成本', fontsize=14, fontweight='bold')
        ax.set_ylabel('成本 (CNY)', fontsize=12)
        ax.set_xlabel('版本', fontsize=12)
        ax.grid(axis='y', alpha=0.3)
        
        # 添加数值标签
        for i, v in enumerate(df_benchmarks['cost_total_cost_cny']):
            ax.text(i, v + 0.0001, f'¥{v:.4f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('cost_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ 成本对比图表已生成")

## 📋 汇总对比表

In [ ]:
if not df_benchmarks.empty and len(df_benchmarks) >= 2:
    # 计算改进百分比
    baseline = df_benchmarks.iloc[0]
    latest = df_benchmarks.iloc[-1]
    
    improvements = []
    
    metrics = [
        ('perf_end_to_end_avg_time', '端到端时间', 's', 'lower'),
        ('perf_llm_avg_response_time', 'LLM 响应时间', 's', 'lower'),
        ('perf_data_fetching_avg_time', '数据获取时间', 's', 'lower'),
        ('cost_avg_total_tokens', 'Token 使用', '个', 'lower'),
        ('cost_total_cost_cny', '单次成本', '元', 'lower'),
        ('quality_format_correct_rate', '格式正确率', '%', 'higher'),
        ('quality_content_complete_rate', '内容完整率', '%', 'higher'),
    ]
    
    for metric, name, unit, direction in metrics:
        if metric in baseline and metric in latest:
            base_val = baseline[metric]
            latest_val = latest[metric]
            
            if base_val > 0:
                change_pct = ((latest_val - base_val) / base_val) * 100
                
                # 判断是改进还是退步
                if direction == 'lower':
                    is_improvement = change_pct < 0
                    emoji = '✅' if is_improvement else '❌'
                else:
                    is_improvement = change_pct > 0
                    emoji = '✅' if is_improvement else '❌'
                
                improvements.append({
                    '指标': name,
                    f'{baseline["version"]}': f'{base_val:.4f} {unit}',
                    f'{latest["version"]}': f'{latest_val:.4f} {unit}',
                    '变化': f'{change_pct:+.1f}%',
                    '状态': emoji
                })
    
    df_improvements = pd.DataFrame(improvements)
    
    print("\n" + "="*80)
    print(f"📊 优化效果总结: {baseline['version']} → {latest['version']}")
    print("="*80)
    display(df_improvements)
    
    # 保存为 CSV
    df_improvements.to_csv('improvement_summary.csv', index=False, encoding='utf-8-sig')
    print("\n✅ 汇总表已保存: improvement_summary.csv")
    
elif len(df_benchmarks) == 1:
    print("ℹ️  只有一个版本的数据，无法对比")
else:
    print("ℹ️  暂无数据")

## 💡 优化建议

In [ ]:
if not df_benchmarks.empty:
    latest = df_benchmarks.iloc[-1]
    
    recommendations = []
    
    # 性能建议
    if 'perf_llm_avg_response_time' in latest and latest['perf_llm_avg_response_time'] > 3.0:
        recommendations.append({
            '类型': '性能',
            '优先级': '高',
            '建议': 'LLM 响应时间 > 3s，建议优化 Prompt 或尝试更快的模型'
        })
    
    if 'perf_data_fetching_avg_time' in latest and latest['perf_data_fetching_avg_time'] > 1.0:
        recommendations.append({
            '类型': '性能',
            '优先级': '中',
            '建议': '数据获取时间 > 1s，建议实现缓存机制'
        })
    
    # 质量建议
    if 'quality_content_complete_rate' in latest and latest['quality_content_complete_rate'] < 0.9:
        recommendations.append({
            '类型': '质量',
            '优先级': '高',
            '建议': '内容完整率 < 90%，建议优化 Prompt 指令的明确性'
        })
    
    # 成本建议
    if 'cost_avg_total_tokens' in latest and latest['cost_avg_total_tokens'] > 2000:
        recommendations.append({
            '类型': '成本',
            '优先级': '中',
            '建议': 'Token 使用量较高，建议精简 Prompt 或使用更小的模型'
        })
    
    if recommendations:
        print("\n" + "="*80)
        print("💡 优化建议")
        print("="*80)
        df_recommendations = pd.DataFrame(recommendations)
        display(df_recommendations)
    else:
        print("\n✅ 所有指标表现良好，无需优化！")

## 💾 导出报告

In [ ]:
# 生成 Markdown 报告
if not df_benchmarks.empty:
    report_lines = [
        "# 📊 优化效果报告\n",
        f"**生成时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n",
        f"**对比版本**: {df_benchmarks.iloc[0]['version']} → {df_benchmarks.iloc[-1]['version']}\n\n",
        "## 性能改进\n",
        "![性能对比](performance_comparison.png)\n\n",
        "## 质量改进\n",
        "![质量对比](quality_comparison.png)\n\n",
        "## 成本改进\n",
        "![成本对比](cost_comparison.png)\n\n",
    ]
    
    if len(df_benchmarks) >= 2:
        report_lines.append("## 汇总数据\n\n")
        report_lines.append(df_improvements.to_markdown(index=False))
    
    with open('optimization_report.md', 'w', encoding='utf-8') as f:
        f.writelines(report_lines)
    
    print("✅ 报告已导出: optimization_report.md")